In [1]:
import os
import yfinance as yf
from duckduckgo_search import DDGS

# from groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import ServiceContext, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.groq import Groq
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.base.llms.types import MessageRole, ChatMessage
from llama_index.core.schema import Document

from llama_parse import LlamaParse

from transformers import BitsAndBytesConfig

import torch

import nest_asyncio
nest_asyncio.apply()

LLAMA_CLOUD_API_KEY = os.environ.get('LLAMAPARSE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
OPENAI_API_KEY= os.environ.get('OPENAI_API_KEY')
CLAUDE_API_KEY = os.environ.get('CLAUDE_API_KEY')

/home/reza/anaconda3/envs/fico/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_query_engine_tool_from_md(md: str, tool_name: str, tool_description: str):
    doc_ = Document.from_dict({'text': md})
    index_ = VectorStoreIndex.from_documents([doc_])
    
    query_engine_ = index_.as_query_engine(response_mode="compact")

    query_engine_tool_ = QueryEngineTool(
        query_engine=query_engine_,
        metadata=ToolMetadata(
            name=tool_name,
            description=(tool_description),
        ),
    )
        
    return query_engine_, query_engine_tool_

In [3]:
ticker_to_research = "BBCA.JK"
ticker = yf.Ticker(ticker_to_research)

In [4]:
stock_name = ticker_to_research.lower().replace(".", "_")
stock_name

'bbca_jk'

In [5]:
tools = []

In [6]:
company_info_full = ticker.info
if company_info_full:
    company_info_cleaned = {
        "Name": company_info_full.get("shortName"),
        "Symbol": company_info_full.get("symbol"),
        "Current Stock Price": f"{company_info_full.get('regularMarketPrice', company_info_full.get('currentPrice'))} {company_info_full.get('currency', 'USD')}",
        "Market Cap": f"{company_info_full.get('marketCap', company_info_full.get('enterpriseValue'))} {company_info_full.get('currency', 'USD')}",
        "Sector": company_info_full.get("sector"),
        "Industry": company_info_full.get("industry"),
        "Address": company_info_full.get("address1"),
        "City": company_info_full.get("city"),
        "State": company_info_full.get("state"),
        "Zip": company_info_full.get("zip"),
        "Country": company_info_full.get("country"),
        "EPS": company_info_full.get("trailingEps"),
        "P/E Ratio": company_info_full.get("trailingPE"),
        "52 Week Low": company_info_full.get("fiftyTwoWeekLow"),
        "52 Week High": company_info_full.get("fiftyTwoWeekHigh"),
        "50 Day Average": company_info_full.get("fiftyDayAverage"),
        "200 Day Average": company_info_full.get("twoHundredDayAverage"),
        "Website": company_info_full.get("website"),
        "Summary": company_info_full.get("longBusinessSummary"),
        "Analyst Recommendation": company_info_full.get("recommendationKey"),
        "Number Of Analyst Opinions": company_info_full.get("numberOfAnalystOpinions"),
        "Employees": company_info_full.get("fullTimeEmployees"),
        "Total Cash": company_info_full.get("totalCash"),
        "Free Cash flow": company_info_full.get("freeCashflow"),
        "Operating Cash flow": company_info_full.get("operatingCashflow"),
        "EBITDA": company_info_full.get("ebitda"),
        "Revenue Growth": company_info_full.get("revenueGrowth"),
        "Gross Margins": company_info_full.get("grossMargins"),
        "Ebitda Margins": company_info_full.get("ebitdaMargins"),
    }
    company_info_md = "## Company Info\n\n"
    for key, value in company_info_cleaned.items():
        if value:
            company_info_md += f"  - {key}: {value}\n\n"
    
    description = f"""Provides current company information about {stock_name}.
        Use a detailed plain text question as input to the tool."""
    query_engine_company_info, tools_company_info = create_query_engine_tool_from_md(company_info_md, stock_name+"_company_info", description)
    tools.append(tools_company_info)

In [7]:
ddgs = DDGS()
company_news = ddgs.news(keywords=ticker_to_research+" stocks", max_results=5)
if len(company_news) > 0:
    company_news_md = "## Company News\n\n\n"
    for news_item in company_news:
        company_news_md += f"#### {news_item['title']}\n\n"
        if "date" in news_item:
            company_news_md += f"  - Date: {news_item['date']}\n\n"
        if "url" in news_item:
            company_news_md += f"  - Link: {news_item['url']}\n\n"
        if "source" in news_item:
            company_news_md += f"  - Source: {news_item['source']}\n\n"
        if "body" in news_item:
            company_news_md += f"{news_item['body']}"
        company_news_md += "\n\n"
    
    description = f"""Provides 5 current news about {stock_name}.
        Use a detailed plain text question as input to the tool."""
    query_engine_company_news, tools_company_news = create_query_engine_tool_from_md(company_news_md, stock_name+"_company_news", description)
    tools.append(tools_company_news)

In [8]:
query_engine_company_news.query(f"Summarize the news about {ticker_to_research} that give insights about the stock performance")

Response(response="The news about BBCA.JK provides insights into the stock's valuation through the Price to Earnings (P/E) ratio, highlighting a key measure for assessing the stock's performance in relation to its earnings per share. Additionally, the risk disclosure associated with trading in financial instruments and cryptocurrencies emphasizes the high risks involved, indicating a cautious approach may be necessary when considering investments in BBCA.JK.", source_nodes=[NodeWithScore(node=TextNode(id_='599707ef-ee38-42a3-9ae4-3e34faa13970', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7e401ef2-15fd-489e-9b2a-610927e6cf0f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='75e0b8a0b8bdce4f77b8959c023b9843205aa9d19e4f856c02e839b6468c5e19')}, text="## Company News\n\n\n#### Bank Central Asia Tbk (BBCA)\n\n  - Date: 2024-05-03T09:14:00+00:00\n\n  - Link: https:

In [9]:
analyst_recommendations = ticker.recommendations
if not analyst_recommendations.empty:
    analyst_recommendations_md = analyst_recommendations.to_markdown()

    description = f"""Provides analyst recommendation about {stock_name}.
        Use a detailed plain text question as input to the tool."""
    query_engine_analyst_recommendations, tools_analyst_recommendations = create_query_engine_tool_from_md(analyst_recommendations_md, stock_name+"_analyst_recommendations", description)
    tools.append(tools_analyst_recommendations)

In [10]:
query_engine_analyst_recommendations.query(f"Explain the analyst recomendation about {ticker_to_research}?")

Response(response='The analyst recommendation about BBCA.JK is mostly leaning towards a positive outlook, with a majority of analysts suggesting either a buy or hold rating. There are very few sell recommendations, indicating a general sentiment of confidence in BBCA.JK.', source_nodes=[NodeWithScore(node=TextNode(id_='eafbd021-2d55-42b3-b82b-e7c6aff45098', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='811fea5a-3be5-4929-8457-87d7dc5ba20b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='4c1de6451ec8ebe363ffc5a4f1de91af6d690d9a6ee80a4b00f61ba612716be0')}, text='|    | period   |   strongBuy |   buy |   hold |   sell |   strongSell |\n|---:|:---------|------------:|------:|-------:|-------:|-------------:|\n|  0 | 0m       |           0 |     0 |      0 |      0 |            0 |\n|  1 | -1m      |           2 |     7 |     10 |      2 |            1 |\n|  2 | -

In [11]:
upgrades_downgrades = ticker.upgrades_downgrades[0:20]
if not upgrades_downgrades.empty:
    upgrades_downgrades_md = upgrades_downgrades.to_markdown()

    description = f"""Provides upgrades and downgrades of {stock_name}.
        Use a detailed plain text question as input to the tool."""
    query_engine_upgrades_downgrades, tools_upgrades_downgrades = create_query_engine_tool_from_md(upgrades_downgrades_md, stock_name+"_upgrades_downgrades_md", description)
    tools.append(tools_upgrades_downgrades)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BBCA.JK?modules=upgradeDowngradeHistory&corsDomain=finance.yahoo.com&formatted=false&symbol=BBCA.JK&crumb=mDLMZyBHZKz


In [12]:
local_path='./data/'
path_pdf = local_path + 'bbca_split.pdf'

output_directory_path = local_path+'/test'

In [13]:
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

# llm_local = HuggingFaceLLM(
#     model_name="meta-llama/Llama-2-7b-chat-hf",
#     tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
#     query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
#     context_window=3900,
#     model_kwargs={"quantization_config": quantization_config},
#     device_map="auto",
# )

In [14]:
# llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY, is_function_calling_model=True)
llm_anthropic = Anthropic(model="claude-3-haiku-20240307", api_key=CLAUDE_API_KEY)
# llm_anthropic = Anthropic(model="claude-3-opus-20240229", api_key=CLAUDE_API_KEY)
# llm_openai = OpenAI(model="gpt-4-turbo", temperature=0.1)

Settings.llm = llm_anthropic
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

In [15]:
# set up parser
parser = LlamaParse(api_key=LLAMA_CLOUD_API_KEY, result_type="markdown")

In [16]:
file_extractor = {".pdf": parser}
dir_reader = SimpleDirectoryReader(input_files=[path_pdf], file_extractor=file_extractor)
annual_report_documents = dir_reader.load_data(show_progress=True)

Loading files:   0%|          | 0/1 [00:00<?, ?file/s]

Started parsing the file under job_id e89ccf84-d244-400c-817f-0a4e00a5d6e2


Loading files: 100%|██████████| 1/1 [00:02<00:00,  2.84s/file]


In [17]:
# Index the documents in a vectorStore
annual_report_index = VectorStoreIndex.from_documents(annual_report_documents)
query_engine_annual_report = annual_report_index.as_query_engine(response_mode="compact")

In [18]:
# query the engine
query = "What was the Net Income in 2022, the Reported one?"
response_jpmorgan = query_engine_annual_report.query(query)
response_jpmorgan.response

'The Net Income in 2022 was 40,756 billion Rupiah, as reported in the financial highlights table.'

In [29]:
# query the engine
query = "Summarize what information included in the annual report? write in bullet points."
query_engine_annual_report.query(query).response

'- The annual report provides detailed financial information for PT Bank Central Asia Tbk (BCA) over the past 5 years, including:\n\n- Financial position data such as total assets, earning assets, loans, placements, liabilities, third-party funds, equity, etc.\n\n- Comprehensive income data such as operating income, net interest income, operating expenses, impairment losses, net income, and total comprehensive income.\n\n- Breakdowns of key financial components like current accounts, savings accounts, time deposits, borrowings, and debt securities.\n\n- Earnings per share information.\n\n- Notes on the financial data, such as adjustments for the 1:5 stock split in 2021 and reporting in accordance with regulatory guidelines.\n\n- The report covers both the consolidated group results as well as the parent company-only figures.'

In [20]:
#Create the Query Engine Tool over the RAG pipeline
query_engine_tool_ar = QueryEngineTool(
    query_engine=query_engine_annual_report,
    metadata=ToolMetadata(
        name=stock_name+"_annualreport",
        description=(
        f"""Provides Annual Report about {stock_name}
        Use a detailed plain text question as input to the tool."""
        ),
    ),
)

tools.append(query_engine_tool_ar)

In [21]:
def evolution(a: float, b: float) -> float:
    """Evolution from value b to value a"""
    return f"{a-b}"
evolution_tool = FunctionTool.from_defaults(fn=evolution)

def evolution_perc(a: float, b: float) -> float:
    """Evolution in percentage from value b to value a"""
    return f"{round(100*(a/b-1),0)}%"
evolution_perc_tool = FunctionTool.from_defaults(fn=evolution_perc)

def cagr(a: float, b: float, n: int) -> float:
    """Compound annual growth rate CAGR from value b to value a over n period"""
    return f"{round(100*((a/b)**(1/n)-1),0)}%"
cagr_tool = FunctionTool.from_defaults(fn=cagr)

def price_earning_ratio(price: float, eps: float) -> float:
    """Price per earning ratio is a ratio of the share price devided by the earning per share(EPS) value"""
    return price/eps
pe_tool = FunctionTool.from_defaults(fn=price_earning_ratio)

# def is_document_available() -> bool:
#     """Check if the document is available"""
#     return documents is not None
# is_document_available_tool = FunctionTool.from_defaults(fn=is_document_available)


In [22]:
system_prompt = f"""You are a usefull asstistant for financial analysis. Answer the question related to that company. 
Use the {stock_name+"_annualreport"} tool to get information about annual report, for example: net income, share performance, capital, etc.
Use the {stock_name+"_company_info"} tool to get current information of the stock or company like company name, current stock price, etc.
Use the {stock_name+"_company_news"} tool to get news about the company, you can ask summary of the news using this tools.
Use the {stock_name+"_analyst_recommendations"} tool to get analyst recommendations, you can ask insight from analyst recomandations about the stocks.
Use the {stock_name+"_upgrades_downgrades_md"} tool to get upgrades and downgrades of the stocks.
Carefully read this information and provide a clear and concise answer to the user.
When you share numbers, make sure to include the units (e.g., millions/billions) and currency.
Do not use phrases like 'based on my knowledge' or 'depending on the information'.
do not answer any question that is not related to the {ticker_to_research}."""

In [23]:
agent_worker = FunctionCallingAgentWorker.from_tools(
        tools+[evolution_tool, evolution_perc_tool, cagr_tool, pe_tool], llm=llm_anthropic,
        system_prompt=system_prompt,
        verbose=True,
    )
agent = agent_worker.as_agent()

In [24]:
# chat_history = [ChatMessage(role=MessageRole.SYSTEM, content=system_prompt)]

In [26]:
query = "What was the highest and lowest stock price in 2022, the Reported one?"
response = agent.chat(query)
print(str(response))

Added user message to memory: What was the highest and lowest stock price in 2022, the Reported one?
=== Calling Function ===
Calling function: bbca_jk_annualreport with args: {"input": "bbca_jk highest and lowest stock price in 2022"}
=== Function Output ===
According to the management report provided in the context, the highest and lowest stock prices for BCA (bbca_jk) in 2022 were:

Highest price in 2022: 9,400 Rupiah
Lowest price in 2022: 7,000 Rupiah

The report shows the stock price movements for BCA from 2019 to 2023, with the highest and lowest prices for each quarter. In 2022, the highest price was 9,400 Rupiah in the fourth quarter, and the lowest price was 7,000 Rupiah in the third quarter.
=== LLM Response ===
The highest reported stock price for bbca_jk in 2022 was 9,400 Rupiah, and the lowest reported stock price was 7,000 Rupiah.
assistant: The highest reported stock price for bbca_jk in 2022 was 9,400 Rupiah, and the lowest reported stock price was 7,000 Rupiah.


In [24]:
query = "What is the analyst recomandation about this stocks?"
response = agent.chat(query)
print(str(response))

assistant: Based on the analyst recommendations for bbca_jk:

- The majority of analysts currently have a "Buy" rating on the stock, indicating they believe the stock has potential for further upside.
- There are also a number of analysts with a "Hold" rating, suggesting the stock is trading at a fair valuation currently and may not have significant near-term upside or downside.
- Overall, the analyst consensus seems to be that bbca_jk is a reasonably attractive investment opportunity, with the potential for the stock price to appreciate over time. However, there are also some analysts who believe the current valuation is appropriate and the stock may not see major gains in the short-term.

The analyst recommendations provide a mixed but generally positive outlook on the prospects for bbca_jk going forward. Investors should review the full range of analyst views to form their own assessment of the company and stock.


In [25]:
query = "What was the Net Income in 2020, the Reported one?"
response = agent.chat(query)
print(str(response))

assistant: The reported Net Income for bbca_jk in 2020 was 27,147 billion Indonesian Rupiah.


In [26]:
MessageRole.USER.value

'user'

In [27]:
[{"role": chat.role.value, "content": chat.content} for chat in agent.chat_history if chat.role in [MessageRole.USER, MessageRole.ASSISTANT] and len(chat.additional_kwargs.get('tool_calls', [])) == 0]

[{'role': 'user',
  'content': 'What is the analyst recomandation about this stocks?'},
 {'role': 'assistant',
  'content': 'Based on the analyst recommendations for bbca_jk:\n\n- The majority of analysts currently have a "Buy" rating on the stock, indicating they believe the stock has potential for further upside.\n- There are also a number of analysts with a "Hold" rating, suggesting the stock is trading at a fair valuation currently and may not have significant near-term upside or downside.\n- Overall, the analyst consensus seems to be that bbca_jk is a reasonably attractive investment opportunity, with the potential for the stock price to appreciate over time. However, there are also some analysts who believe the current valuation is appropriate and the stock may not see major gains in the short-term.\n\nThe analyst recommendations provide a mixed but generally positive outlook on the prospects for bbca_jk going forward. Investors should review the full range of analyst views to fo

In [28]:
agent.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the analyst recomandation about this stocks?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='', additional_kwargs={'tool_calls': [{'id': 'toolu_012ym1CFZmU7gg4Sz9TuvtjA', 'input': {'input': 'analyst recommendation about bbca_jk'}, 'name': 'bbca_jk_analyst_recommendations', 'type': 'tool_use'}]}),
 ChatMessage(role=<MessageRole.TOOL: 'tool'>, content='The analyst recommendation about bbca_jk is generally positive, with a mix of buy and hold ratings.', additional_kwargs={'name': 'bbca_jk_analyst_recommendations', 'tool_call_id': 'toolu_012ym1CFZmU7gg4Sz9TuvtjA'}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Based on the analyst recommendations for bbca_jk:\n\n- The majority of analysts currently have a "Buy" rating on the stock, indicating they believe the stock has potential for further upside.\n- There are also a number of analysts with a "Hold" rating, suggesti

In [29]:
query = "What was the evolution of the Net Income from 2020 to 2021, the Reported one?"
response = agent.chat(query)
print(str(response))

assistant: The reported Net Income of bbca_jk increased from 27,147 billion Rupiah in 2020 to 31,440 billion Rupiah in 2021, an evolution of 4,293 billion Rupiah or approximately 15.8% growth year-over-year.


In [30]:
query = "What is the current stock price of BBCA.JK?"
response = agent.chat(query)
print(str(response))

assistant: According to the company information provided, the current stock price of BBCA.JK is 9850.0 Indonesian Rupiah.


In [31]:
agent.memory.chat_store.store['chat_history'][0].role in [MessageRole.USER, MessageRole.ASSISTANT]

True

In [32]:
[chat for chat in agent.memory.chat_store.store['chat_history'] if chat.role in [MessageRole.USER, MessageRole.ASSISTANT] and len(chat.additional_kwargs.get('tool_calls', [])) == 0]

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the analyst recomandation about this stocks?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Based on the analyst recommendations for bbca_jk:\n\n- The majority of analysts currently have a "Buy" rating on the stock, indicating they believe the stock has potential for further upside.\n- There are also a number of analysts with a "Hold" rating, suggesting the stock is trading at a fair valuation currently and may not have significant near-term upside or downside.\n- Overall, the analyst consensus seems to be that bbca_jk is a reasonably attractive investment opportunity, with the potential for the stock price to appreciate over time. However, there are also some analysts who believe the current valuation is appropriate and the stock may not see major gains in the short-term.\n\nThe analyst recommendations provide a mixed but generally positive outlook on the prospects for bbca_jk goi

In [33]:
query = "What was the evolution of the Net Income from 2021 to 2022, the Reported one?"
response = agent.chat(query)
print(str(response))

assistant: The reported Net Income of bbca_jk increased from 31,440 billion Rupiah in 2021 to 40,756 billion Rupiah in 2022, an evolution of 9,316 billion Rupiah or approximately 29.6% growth year-over-year.


In [34]:
query = "What was the evolution in term of percentage of the Net Income from 2020 to 2021, the Reported one?"
response = agent.chat(query)
print(str(response))

assistant: The reported Net Income of bbca_jk increased from 27,147 billion Rupiah in 2020 to 31,440 billion Rupiah in 2021, representing an evolution of 16.0% in percentage terms.


In [35]:
query = "Compare the evolution of the Net Income between 2020 to 2021 and 2021 to 2022, the Reported one."
response = agent.chat(query)
print(str(response))

assistant: Okay, let's compare the evolution of the reported Net Income for bbca_jk between 2020 to 2021 and 2021 to 2022:

2020 to 2021 Net Income evolution:
- 2020 Net Income: 27,147 billion Rupiah
- 2021 Net Income: 31,440 billion Rupiah
- Evolution from 2020 to 2021: 16.0% increase

2021 to 2022 Net Income evolution: 
- 2021 Net Income: 31,440 billion Rupiah
- 2022 Net Income: 40,756 billion Rupiah
- Evolution from 2021 to 2022: 29.6% increase

The key differences are:

- From 2020 to 2021, the Net Income grew by 16.0%
- From 2021 to 2022, the Net Income grew at a faster pace of 29.6%

So the Net Income growth accelerated from 2020-2021 to 2021-2022, increasing from 16.0% to 29.6% year-over-year. This indicates the company's profitability and earnings power strengthened more significantly in the most recent year compared to the prior year.

Overall, the reported Net Income for bbca_jk showed strong growth in both periods, but the pace of growth picked up notably from 2021 to 2022.


In [36]:
query = "What was the compound annual growth of the Net Income from 2020 to 2022, the Reported one?"
response = agent.chat(query)
print(str(response))

assistant: The compound annual growth rate (CAGR) of the reported Net Income for bbca_jk from 2020 to 2022 was 23.0%.

This indicates the Net Income grew at an average annual rate of 23.0% over this 2-year period, which is a strong and accelerating growth trajectory for the company.


In [38]:
query = "based on the lowest and highest stock price of BBCA.JK in 2023 and given the financial report, what was the price/earning ratio?"
response = agent.chat(query)
print(str(response))

assistant: So the current price-to-earnings (P/E) ratio for bbca_jk based on the 2022 Net Income of 40,756 billion Rupiah and the current stock price of 9850.0 Indonesian Rupiah is approximately 0.24.

This relatively low P/E ratio suggests the stock may be undervalued compared to the company's earnings power. However, more information would be needed to fully assess the valuation.


In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

In [ ]:
query_engine = RouterQueryEngine.from_defaults(
    [query_engine_tool],
    select_multi=True,
    verbose=True,
)

In [ ]:
query = "What was the Net Income in 2020, the Reported one?"
response = query_engine.query(query)
response.response

Selecting query engine 0: This choice is relevant because it provides Annual Report information about JPMorgan, which may include the Net Income for 2020..


'The Net Income in 2020, the Reported one, was $29.1 billion.'

In [ ]:
query = "What was the evolution of the Net Income from 2020 to 2021, the Reported one?"
response = query_engine.query(query)
response.response

Selecting query engine 0: This choice is relevant because it provides Annual Report information about JPMorgan for 2022, which may include the Net Income data for 2020 and 2021, allowing us to determine the evolution of the Net Income from 2020 to 2021..


'The Net Income evolved from $29.1 billion in 2020 to $48.3 billion in 2021.'

In [ ]:
query = "What was the evolution in term of percentage of the Net Income from 2020 to 2021, the Reported one?"
response = query_engine.query(query)
response.response

Selecting query engine 0: This choice is relevant because it provides Annual Report information about JPMorgan for 2022, which likely includes financial data such as Net Income for previous years, allowing for the calculation of the percentage evolution from 2020 to 2021..


'The Net Income in 2020 was $29.1 billion, and in 2021 it was $48.3 billion. To find the percentage change, first, find the difference: $48.3 billion - $29.1 billion = $19.2 billion. Then, divide the difference by the initial value and multiply by 100: ($19.2 billion / $29.1 billion) * 100 = 65.9%.'